In [ ]:
# Just run this block. Please do not modify the following code.
import math
import time
import io
import numpy as np
import csv
from IPython.display import Image

# Pytorch package
import torch
import torch.nn as nn
import torch.optim as optim

# Torchtest package
# import torchtext
# from torchtext.datasets import Multi30k
from torch.utils.data import DataLoader
# from torchtext.data.utils import get_tokenizer
from collections import Counter
# from torchtext.vocab import vocab
# from torchtext.utils import download_from_url, extract_archive
from torch.nn.utils.rnn import pad_sequence
from torch.optim import Adam

# Tqdm progress bar
from tqdm import tqdm_notebook, tqdm

# Code provide to you for training and evaluation

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

torch.seed

In [ ]:
# Check device availability
device = ''
if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = 'mps'
else:
    device = 'cpu'
device = torch.device(device)
print("You are using device: %s" % device)

# Training, evaluation, and plotting loss curves

In [ ]:
# Code provide to you for training and evaluation
from utils_experiment2 import train
from utils_experiment2 import evaluate
from utils_experiment2 import plot_curves
from Model import Model

def train_and_plot(model, optimizer, scheduler, criterion, train_loader, valid_loader, filename, epochs, device='cpu'):
    train_loss_history = []
    valid_loss_history = []

    for epoch_idx in range(epochs):
        print("-----------------------------------")
        print("Epoch %d" % (epoch_idx+1))
        print("-----------------------------------")

        train_loss, avg_train_loss = train(model, train_loader, optimizer, criterion, device=device)
        scheduler.step(train_loss)

        val_loss, avg_val_loss = evaluate(model, valid_loader, criterion, device=device)

        train_loss_history.append(avg_train_loss)
        valid_loss_history.append(avg_val_loss)

        print("Training Loss: %.4f. Validation Loss: %.4f." % (avg_train_loss, avg_val_loss))

    plot_curves(train_loss_history, valid_loss_history, filename)


In [ ]:
from dataset import preppedData 
import numpy as np
# Example usage:
model = Model().to(device)


model.load_state_dict(torch.load('model_final_e2.pt'),strict=False)

spec = np.load('./spectrograms_v2.npy')
style = np.load('sing_resemblyzer_vectors_v2.npy')
target = np.load('speech_resemblyzer_vectors.npy')

spec = np.transpose(spec,(0,2,1))#use if the dataset is not already transposed
print(spec.shape)
style = style[0:spec.shape[0],:]
target = np.concatenate((target,target),axis=0)
print(target.shape)
dataList = []

style = style[0:target.shape[0],:]

spec = spec[0:target.shape[0],:,:]

for i in range(0,spec.shape[0]):
    dataList.append((spec[i,:,:],style[i,:],target[i,:]))

print(len(dataList))

print(dataList[0][0].shape)
print(dataList[0][1].shape)
print(dataList[0][2].shape)


In [8]:
from utils_experiment2 import criterion


pct_2_use = 0.5
train_pct = 0.8

chunk_end = int(len(dataList)*pct_2_use)

train_end_ind = int(round(chunk_end*train_pct,0))

print(train_end_ind)

your_train_dataset = dataList[0:train_end_ind]

your_valid_dataset = dataList[train_end_ind:chunk_end]
print(len(your_train_dataset))
print(len(your_valid_dataset))
print(your_train_dataset[0][0].shape)


2145
2145
536
(256, 692)


In [9]:

train_loader = DataLoader(your_train_dataset, batch_size=5, shuffle=True)
valid_loader = DataLoader(your_valid_dataset, batch_size=5, shuffle=False)
optimizer = Adam(model.parameters(), lr=5e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')
criterion = criterion
filename = "training_loss_curves"
epochs = 1
# device = 'cuda' if torch.cuda.is_available() else 'cpu'

train_and_plot(model, optimizer, scheduler, criterion, train_loader, valid_loader, filename, epochs, device)

-----------------------------------
Epoch 1
-----------------------------------


Training:   0%|          | 0/429 [00:00<?, ?it/s]


TypeError: reconstruct() takes from 3 to 4 positional arguments but 5 were given

In [7]:
torch.save(model.state_dict(),'model_final_e2_w_styleV2.pt')